In [1]:
import pickle
from helper_fun import get_evaluator_info, evaluate_circ, apply_measurement

device_name = 'ibmq_boeblingen'
job_submittor_input = pickle.load(open('./benchmark_data/job_submittor_input_ibmq_boeblingen_saturated.p', 'rb' ))
case = (4,6)
cluster_idx = 0
cluster_circ = job_submittor_input[case]['clusters'][cluster_idx]

evaluator_info = get_evaluator_info(circ=cluster_circ,device_name=device_name,
fields=['device','basis_gates','coupling_map','properties','initial_layout','noise_model','num_shots','meas_filter'])

Calculating measurement filter, 20-qubit calibration circuits * 2 * 3.072e+03 shots. 1.438e+02 seconds


In [7]:
from qiskit.compiler import transpile, assemble
def submit_hardware_jobs(circs, evaluator_info):
    if evaluator_info['num_shots']>evaluator_info['device'].configuration().max_shots:
        print('During circuit evaluation on hardware, num_shots %.3e exceeded hardware max'%evaluator_info['num_shots'])
        evaluator_info['num_shots'] = device.configuration().max_shots
    
    mapped_circuits = []
    for circ in circs:
        qc=apply_measurement(circ)
        mapped_circuit = transpile(qc,
        backend=evaluator_info['device'], basis_gates=evaluator_info['basis_gates'],
        coupling_map=evaluator_info['coupling_map'],backend_properties=evaluator_info['properties'],
        initial_layout=evaluator_info['initial_layout'])
        mapped_circuits.append(mapped_circuit)

    qobj = assemble(mapped_circuits, backend=evaluator_info['device'], shots=evaluator_info['num_shots'])
    job = evaluator_info['device'].run(qobj)
    
    hw_result = job.result()
    return hw_result

In [8]:
cluster_instances = job_submittor_input[case]['all_cluster_prob'][cluster_idx]
print('Cluster %d has %d instances'%(cluster_idx,len(cluster_instances)))

hw_result = submit_hardware_jobs(circs=cluster_instances.values(),evaluator_info=evaluator_info)

Cluster 0 has 9 instances


In [18]:
mapped_circuits = []
for circ in cluster_instances.values():
    qc=apply_measurement(circ)
    mapped_circuit = transpile(qc,
    backend=evaluator_info['device'], basis_gates=evaluator_info['basis_gates'],
    coupling_map=evaluator_info['coupling_map'],backend_properties=evaluator_info['properties'],
    initial_layout=evaluator_info['initial_layout'])
    mapped_circuits.append(mapped_circuit)

In [19]:
mitigated_results = evaluator_info['meas_filter'].apply(hw_result)
hw_counts = [mitigated_results.get_counts(circ) for circ in mapped_circuits]

QiskitError: 'Data for experiment "circuit42" could not be found.'